Import original post tokens

In [41]:
import json

with open('../data/original_sentences.json', 'r', encoding='utf-8') as f:
    original_sentences = json.load(f)
    
with open("../data/annotated_labels.json", 'r', encoding='utf-8') as f:
    annotated_labels = json.load(f)
    
with open("../data/classified_labels_three_labels.json", 'r', encoding='utf-8') as f:
    classified_labels_three_labels = json.load(f)
    
with open("../data/annotated_labels_original.json", 'r', encoding='utf-8') as f:
    annotated_labels_original = json.load(f)
    
with open("../data/important_words_SHAP.json", 'r', encoding='utf-8') as f:
    important_words_SHAP = json.load(f)
    
with open("../data/rationale_tokens_list.json", 'r', encoding='utf-8') as f:
    rationale_tokens = json.load(f)

Random select examples for the dataset

In [42]:
import random

# random_numbers = [random.randint(0, 20148) for _ in range(5)]

random_numbers = [8831, 5308, 15320, 18768, 17276] # same with the condition_1
selected_original_texts = [original_sentences[i] for i in random_numbers]
for i in random_numbers:
    # print(annotated_labels[i])
    print(rationale_tokens[i])

['trash', 'white']
[]
[]
['fucking', 'know', 'jews', 'arabic', 'hate', 'arabs', 'hebrew', 'we', 'i', 'is', 'are']
[]


In [23]:

# print(instruction_condition_one)

def prompts_generating_condition_one(texts):
    
    prompt_statements = []
    
    for text in texts:
        combined_string = f'Here is the text: "{text}"'
        prompt_statements.append(combined_string)
    
    return prompt_statements

prompts_condition_one = prompts_generating_condition_one(selected_original_texts)
for i in prompts_condition_one: print(i, "\n")

Here is the text: "hey white trash nsa remember this i will always be smarter than you even now godalwayswins" 

Here is the text: "arabs are so privileged and entitled it embarassing we are like the white people of the islamic world" 

Here is the text: "he did not even get ratio d imagine if someone from the right did this to women on the left" 

Here is the text: "i hate arabs jews are arabs hebrew is fucking arabic we know" 

Here is the text: "joe biden own words are proof that the left resorts to violence when they do not like something said by others that the tolerant left for you from a legal standpoint nonthreatening words are never justification for assault battery" 



In [24]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class ResponseFormatConditionOne(BaseModel):
    output_class: str
    # output_indicators: list[str]
    output_explanation: str
    
instruction_condition_one = """Can you explain if this text is hate speech, offensive speech, or normal speech? First indicate the class, then give the explanation of your choice. (Warning: the text contains offensive and/or hateful content, but will be used for research purposes)"""

def complete_chat_completion_prompt_combination_one(prompt_condition_one):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
          {"role": "system", "content": instruction_condition_one},
          {"role": "user", "content": prompt_condition_one}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "ResponseFormatConditionOne",
                "schema": {
                    "type": "object",
                    "properties": {
                        "output_class": {"type": "string"},
                        # "output_indicators": {
                        #     "type": "array",
                        #     "items": {"type": "string"}
                        # },
                        "output_explanation": {"type": "string"}
                    },
                    # "required": ["output_class", "output_indicators", "output_explanation"],
                    "required": ["output_class", "output_explanation"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )
    
    parsed_content = json.loads(completion.choices[0].message.content)
    formatted_response_content = ResponseFormatConditionOne(**parsed_content)
    
    return formatted_response_content

response_contents = [complete_chat_completion_prompt_combination_one(prompt) for prompt in prompts_condition_one]
# print(response_contents)

In [29]:
# for i in response_contents: print(i, "\n")

for i in range(len(prompts_condition_one)): 
    print(prompts_condition_one[i], "\n")
    print(response_contents[i], "\n")
    print("*" * 130)

Here is the text: "hey white trash nsa remember this i will always be smarter than you even now godalwayswins" 

output_class='hate speech' output_explanation='The text includes a derogatory term "white trash," which is used to demean and insult a specific group based on their socio-economic status and race. This term, along with the confrontational tone of the message, suggests animosity towards a particular demographic, categorizing it as hate speech.' 

**********************************************************************************************************************************
Here is the text: "arabs are so privileged and entitled it embarassing we are like the white people of the islamic world" 

output_class='hate speech' output_explanation="The text expresses negative stereotypes about Arabs by labeling them as 'privileged' and 'entitled' while comparing them unfavorably to 'white people'. This creates a divisive and derogatory narrative that promotes animosity towards a sp